In [1]:
# 성능평가 중요성
# 1. 하이퍼 매개변수 최적화나 모델 선택에 중요
# 현장 설치여부를 판단 -- 납품

#객관적인 평가 방법
# 2. 교차검증 : 데이터를 분할해서 여러번 반복해서 통계적으로 신뢰성을 높이

#3 .제거조사 ablation study
# 여러 선택사항이 있을때 선택사항을 하나씩 빼고 성능을 측정해서 최적의 기여도를 평가

# 위의 3가지 옵션에 대한 검증결과를 시각화하고 수치로 보임으로써 신뢰성을 높임

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import KFold
from tensorflow.keras import regularizers

In [3]:
# 데이터 준비
(x_train,y_train),(x_target,y_target) = cifar10.load_data()
x_train = x_train / 255.0 ; x_target = x_target / 255.0
y_train = tf.keras.utils.to_categorical(y_train,10)
y_target = tf.keras.utils.to_categorical(y_target,10)

170508288/170498071 [==============================] - 3s 0us/step


In [4]:
batch_size = 128
n_epoch = 10
k = 5

In [11]:
from tensorflow.keras.losses import categorical_crossentropy
def cross_validation(data_gen,dropout_rate,l2_reg):
  accuracy = []
  for train_index, val_index in  KFold(k).split(x_train):
    xtrain, xval = x_train[train_index], x_train[val_index]  # 학습용 데이터, 검증용 데이터
    ytrain, yval = y_train[train_index], y_train[val_index]  # 학습용의 정답, 검증용의 정답

    # 신경망 설계  C-C-P-D C-C-P-D-FC-D-FC   
    # 32 (3,3)  64(3,3)            512  10
    # relu                              softmax 
    # dropout은 dropout_rate[0]  dropout_rate[1]  dropout_rate[2]
    cnn=Sequential()  # 신경망 객체생성
    cnn.add( Conv2D(32,(3,3), activation='relu',input_shape = (32,32,3) )  )
    cnn.add( Conv2D(32,(3,3), activation='relu' )  )
    cnn.add(MaxPool2D(pool_size=(2,2)))
    cnn.add(Dropout(dropout_rate[0]))

    cnn.add( Conv2D(64,(3,3), activation='relu' )  )
    cnn.add( Conv2D(64,(3,3), activation='relu' )  )
    cnn.add(MaxPool2D(pool_size=(2,2)))
    cnn.add(Dropout(dropout_rate[1]))

    cnn.add(Flatten())                      # 신경망의 입력층
    cnn.add(Dense(512,activation='relu'))   # 신경망의 은닉층
    cnn.add(Dropout(dropout_rate[2]))       # 드랍아웃
    cnn.add(Dense(10,activation='softmax',kernel_regularizer = regularizers.l2(l2_reg) ))   # 신경망의 출력층 # 가중치감소

    # 신경망 모델 학습하고 평가
    cnn.compile(loss = categorical_crossentropy, optimizer = Adam(), metrics=['accuracy'])
    if data_gen:
      generator = ImageDataGenerator(rotation_range=3.0, width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True )
      cnn.fit_generator(generator.flow(xtrain,ytrain,batch_size=batch_size), 
                        epochs=n_epoch,validation_data=( xval,yval ),verbose=1 )
    else:
      cnn.fit(xtrain,ytrain,batch_size=batch_size, 
                        epochs=n_epoch,validation_data=( xval,yval ),verbose=1)   
    accuracy.append(cnn.evaluate(xval,yval))
  return accuracy

In [ ]:
# 하이퍼 매개변수 를 달리해서 신경망을 평가
c_001 = cross_validation(False, [0,0,0], 0.0)
c_002 = cross_validation(True, [0,0,0],0.0)

c_003 = cross_validation(False, [0,0,0],0.1)
c_004 = cross_validation(True, [0,0,0],0.1)

c_005 = cross_validation(False, [0.25,0.25,0.5],0.0)
c_006 = cross_validation(True, [0.25,0.25,0.5],0.0)

c_007 = cross_validation(False, [0.25,0.25,0.5],0.01)
c_008 = cross_validation(True, [0.25,0.25,0.5],0.01)

Epoch 1/10
313/313 [==============================] - 18s 23ms/step - loss: 1.6118 - accuracy: 0.4099 - val_loss: 1.3593 - val_accuracy: 0.5048
Epoch 2/10
313/313 [==============================] - 6s 20ms/step - loss: 1.2328 - accuracy: 0.5603 - val_loss: 1.2021 - val_accuracy: 0.5722
Epoch 3/10
313/313 [==============================] - 6s 21ms/step - loss: 1.0140 - accuracy: 0.6434 - val_loss: 0.9697 - val_accuracy: 0.6503
Epoch 4/10
313/313 [==============================] - 6s 20ms/step - loss: 0.8782 - accuracy: 0.6939 - val_loss: 0.8933 - val_accuracy: 0.6855
Epoch 5/10
313/313 [==============================] - 6s 21ms/step - loss: 0.7650 - accuracy: 0.7330 - val_loss: 0.9279 - val_accuracy: 0.6844
Epoch 6/10
313/313 [==============================] - 6s 20ms/step - loss: 0.6780 - accuracy: 0.7642 - val_loss: 0.8069 - val_accuracy: 0.7214
Epoch 7/10
313/313 [==============================] - 6s 21ms/step - loss: 0.5861 - accuracy: 0.7961 - val_loss: 0.7847 - val_accuracy: 0.732

In [ ]:
print(f"[000](, {cvf0} / {np.array(cvf0).mena()}")



In [ ]:
# 박스 플롯으로 표시
#cvf0 아마도 2차원 배열...
rc_001, rc_002, rc_003, rc_004 = np.array(c_001).reshape(-1),np.array(c_002).reshape(-1),np.array(c_003).reshape(-1),np.array(c_004).reshape(-1)

rc_005, rc_006, rc_007, rc_008 = np.array(c_005).reshape(-1),np.array(c_006).reshape(-1),np.array(c_007).reshape(-1),np.array(c_008).reshape(-1)

In [ ]:
# 박스플롯
